In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime

In [2]:
inflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-inflows.pqt')
outflow = pd.read_parquet('/uss/hdsi-prismdata/q1-ucsd-outflows.pqt')

# remove rows where memo = category
outflow = outflow[outflow['memo'] != outflow['category']]

outflow_ids = set(outflow["prism_consumer_id"].unique())
inflow_ids = set(inflow["prism_consumer_id"].unique())

in_not_out = inflow_ids - outflow_ids
out_not_in = outflow_ids - inflow_ids

#consumers in both inflow and outflow
consumers_both = sorted(set(inflow["prism_consumer_id"]).intersection(outflow["prism_consumer_id"]))

#80-20 train test split
train_ids, test_ids = train_test_split(consumers_both, test_size=0.2, random_state=42)

inflow_train = inflow[inflow["prism_consumer_id"].isin(train_ids)]
inflow_test  = inflow[inflow["prism_consumer_id"].isin(test_ids)]

outflow_train = outflow[outflow["prism_consumer_id"].isin(train_ids)]
outflow_test  = outflow[outflow["prism_consumer_id"].isin(test_ids)]

# print(f'Inflow_train: {inflow_train["amount"].median()}\nInflow_test: {inflow_test["amount"].median()}\nOutflow_train: {outflow_train["amount"].median()}\nOutflow_test: {outflow_test["amount"].median()}')

In [3]:
outflow_train

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category
2,0,acc_0,TST* Casa Del Rio - Exp Fairlawn OH 09/24,18.42,2022-09-26,FOOD_AND_BEVERAGES
4,0,acc_0,Buffalo Wild Wings,26.47,2022-09-12,FOOD_AND_BEVERAGES
6,0,acc_0,Oculus CA 04/16,11.73,2022-04-18,GENERAL_MERCHANDISE
7,0,acc_0,LOS GIRASOLES STOW OH 03/08,30.04,2022-03-09,FOOD_AND_BEVERAGES
8,0,acc_0,BUZZIS LAUNDRY 1 OH 03/28,4.16,2022-03-29,GENERAL_MERCHANDISE
...,...,...,...,...,...,...
2595705,5939,acc_9522,PURCHASE 01-14 PUREKANA XXX-XXXXXXX NJ VNT XXXX,116.58,2023-01-17,GENERAL_MERCHANDISE
2595706,5939,acc_9522,PURCHASE 01-18 WAL-MART #XXXX HUTCHINSON KS S...,12.54,2023-01-18,GROCERIES
2595708,5939,acc_9522,PURCHASE 01-27 CLKBANK*MoonReading XXX-XXX-XXX...,11.00,2023-01-27,GENERAL_MERCHANDISE
2595709,5939,acc_9522,PURCHASE 01-26 CASH APP*DIANA STIE XXXXXXXXXX ...,16.00,2023-01-27,GENERAL_MERCHANDISE


## date attributes
(in date_amnt_feat_funcs.py)


In [4]:
# def create_date_feats(fit_df):
# # create copy of data
#     fit_add = fit_df.copy(deep=True)
#     fit_add['posted_date'] = pd.to_datetime(fit_add['posted_date'])
    
# # day of the week
#     fit_add['day_of_week'] = fit_add['posted_date'].dt.strftime('%a')
    
# # month
#     fit_add['month'] = fit_add['posted_date'].dt.month_name()
    
# # quarter
#     fit_add['quarter'] = fit_add['posted_date'].dt.quarter
#     q_map = {1: 'q1', 2: 'q2', 3: 'q3', 4: 'q4'}
#     fit_add['quarter'] = fit_add['quarter'].map(q_map)
    
# # year
#     fit_add['year'] = fit_add['posted_date'].dt.year
    
# # average time btwn transactions
#     df = fit_add.copy()
#     df = df.sort_values(['prism_consumer_id', 'posted_date'])
    
#     df['days_since_prev'] = df.groupby('prism_consumer_id')['posted_date'].diff().dt.days.fillna(0)
#     df['avg_days_between_txn'] = df.groupby('prism_consumer_id')['days_since_prev'].transform('mean')
    
#     df = df.sort_index() # Restore original order
#     fit_add = df
    
# # rolling avg time btwn transactions (window = 5) --> can be helpful for determining financial stability
#     df = fit_add.sort_values(['prism_consumer_id', 'posted_date']).copy()
    
#     df['rolling_avg_days_between_txn'] = (
#         df.groupby('prism_consumer_id')['days_since_prev']
#           .transform(lambda x: x.rolling(window=5, min_periods=1).mean())
#     )
    
#     fit_add = df.sort_index() # restore to original order
    
# # time since first transaction
#     df = fit_add.sort_values(['prism_consumer_id', 'posted_date']).copy()
#     df['posted_date'] = pd.to_datetime(df['posted_date'])
    
#     first_txn = ( # find first transaction date per customer
#         df.groupby('prism_account_id')['posted_date']
#           .transform('min')
#     )
    
#     df['days_since_first_txn'] = (df['posted_date'] - first_txn).dt.days # compute days since first transaction
    
#     fit_add = df.sort_index() # restore to original order
#     return fit_add

In [5]:
# outflow_train = create_date_feats(outflow_train)

## Amount Attributes
(in date_amnt_feat_funcs.py)

In [6]:
# def create_amnt_feats(fit_df):
#     if 'days_since_first_txn' not in fit_df.columns:
#         fit_df = create_date_feats(outflow_train) # run date func

# # whole dollar amounts
#     fit_df['whole_dollar'] = fit_df['amount'] % 1 == 0
        
# # difference from median amount of transactions per month per customer
#     fit_df['month_med_amnt'] = (fit_df.groupby(['prism_consumer_id', 'year', 'month'])['amount']
#                                            .transform('median')
#                                           )
#     fit_df['month_med_amnt_diff'] = fit_df['amount']-fit_df['month_med_amnt']
    
# # Standard deviation of amounts per consumer
#     group_stats = fit_df.groupby('prism_consumer_id')['amount'].agg(['mean', 'std'])
#     mean,std = group_stats['mean'],group_stats['std']
#     fit_df['amnt_zscore'] = (fit_df['amount'] - mean) / std # compute z-score
#     fit_df['amnt_zscore'] = fit_df['amnt_zscore'].fillna(0) # optional: fill NaN z-scores (e.g. if std = 0 or only one transaction)
    
# # Log-transformed amount --> fix skewness of amounts
#     fit_df['log_amnt'] = np.log1p(fit_df['amount'])
    
#     return fit_df

In [7]:
from date_amnt_feat_funcs import create_date_feats, create_amnt_feats

In [8]:
outflow_train = create_amnt_feats(outflow_train)

In [9]:
outflow_train

,prism_consumer_id,prism_account_id,memo,amount,posted_date,category,day_of_week,month,quarter,year,days_since_prev,avg_days_between_txn,rolling_avg_days_between_txn,days_since_first_txn,whole_dollar,month_med_amnt,month_med_amnt_diff,amnt_zscore,log_amnt
2,0,acc_0,TST* Casa Del Rio - Exp Fairlawn OH 09/24,18.42,2022-09-26,FOOD_AND_BEVERAGES,Mon,September,q3,2022,3.0,1.100917,1.0,224,False,22.66,-4.24,-0.705843,2.966303
4,0,acc_0,Buffalo Wild Wings,26.47,2022-09-12,FOOD_AND_BEVERAGES,Mon,September,q3,2022,6.0,1.100917,1.2,210,False,22.66,3.81,-0.288038,3.313095
6,0,acc_0,Oculus CA 04/16,11.73,2022-04-18,GENERAL_MERCHANDISE,Mon,April,q2,2022,3.0,1.100917,1.4,63,False,26.68,-14.95,0.000000,2.543961
7,0,acc_0,LOS GIRASOLES STOW OH 03/08,30.04,2022-03-09,FOOD_AND_BEVERAGES,Wed,March,q1,2022,1.0,1.100917,0.4,23,False,26.19,3.85,-0.328565,3.435277
8,0,acc_0,BUZZIS LAUNDRY 1 OH 03/28,4.16,2022-03-29,GENERAL_MERCHANDISE,Tue,March,q1,2022,1.0,1.100917,1.0,43,False,26.19,-22.03,0.000000,1.640937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595705,5939,acc_9522,PURCHASE 01-14 PUREKANA XXX-XXXXXXX NJ VNT XXXX,116.58,2023-01-17,GENERAL_MERCHANDISE,Tue,January,q1,2023,8.0,2.002770,5.8,713,False,19.48,97.10,0.000000,4.767119
2595706,5939,acc_9522,PURCHASE 01-18 WAL-MART #XXXX HUTCHINSON KS S...,12.54,2023-01-18,GROCERIES,Wed,January,q1,2023,1.0,2.002770,5.4,714,False,19.48,-6.94,0.000000,2.605648
2595708,5939,acc_9522,PURCHASE 01-27 CLKBANK*MoonReading XXX-XXX-XXX...,11.00,2023-01-27,GENERAL_MERCHANDISE,Fri,January,q1,2023,9.0,2.002770,4.8,723,True,19.48,-8.48,0.000000,2.484907
2595709,5939,acc_9522,PURCHASE 01-26 CASH APP*DIANA STIE XXXXXXXXXX ...,16.00,2023-01-27,GENERAL_MERCHANDISE,Fri,January,q1,2023,0.0,2.002770,4.8,723,True,19.48,-3.48,0.000000,2.833213


In [10]:
# import pandas as pd
import holidays
from datetime import timedelta
from tqdm import tqdm
from pandarallel import pandarallel

# Example: make sure your date column is datetime
# df['posted_date'] = pd.to_datetime(df['posted_date'])

# 1️⃣ Define base holiday calendar
us_holidays = holidays.UnitedStates()

# 2️⃣ Add custom holidays like Black Friday & Cyber Monday
def add_custom_holidays(years):
    custom = {}
    for y in years:
        # Find Thanksgiving (4th Thursday of November)
        thanksgiving = [
            d for d, name in holidays.UnitedStates(years=[y]).items()
            if "Thanksgiving" in name
        ][0]

        # Custom additions
        black_friday = thanksgiving + timedelta(days=1)
        cyber_monday = black_friday + timedelta(days=3)

        custom[black_friday] = "Black Friday"
        custom[cyber_monday] = "Cyber Monday"
    return custom

years = range(outflow_train['posted_date'].dt.year.min(), outflow_train['posted_date'].dt.year.max() + 1)
custom_holidays = add_custom_holidays(years)

# Combine federal + custom holidays
all_holidays = {**us_holidays, **custom_holidays}
holiday_dates = sorted(pd.to_datetime(list(all_holidays.keys())))

# 3️⃣ Helper function to compute distances and names
def holiday_context(date, holiday_dict):
    """Return days to prev/next holiday and their names."""
    sorted_holidays = sorted(holiday_dict.items(), key=lambda x: pd.Timestamp(x[0]))
    holiday_dates = [pd.Timestamp(d) for d, _ in sorted_holidays]
    holiday_names = [n for _, n in sorted_holidays]

    # find holidays before and after this date
    prev_idx = max([i for i, h in enumerate(holiday_dates) if h <= date], default=None)
    next_idx = min([i for i, h in enumerate(holiday_dates) if h >= date], default=None)

    # calculate distances and names
    days_since_prev = (date - holiday_dates[prev_idx]).days if prev_idx is not None else None
    days_until_next = (holiday_dates[next_idx] - date).days if next_idx is not None else None
    prev_name = holiday_names[prev_idx] if prev_idx is not None else None
    next_name = holiday_names[next_idx] if next_idx is not None else None

    return pd.Series({
        'days_since_prev_holiday': days_since_prev,
        'days_until_next_holiday': days_until_next,
        'prev_holiday_name': prev_name,
        'next_holiday_name': next_name
    })

# 4️⃣ Apply to your dataset
# outflow_train[['days_since_prev_holiday', 'days_until_next_holiday',
#     'prev_holiday_name', 'next_holiday_name']] = outflow_train['posted_date'].apply(
#     lambda x: holiday_context(x, all_holidays)
# )



# Initialize pandarallel (enable tqdm progress bar)
pandarallel.initialize(progress_bar=True)

# Parallelized apply across CPU cores
outflow_train[['days_since_prev_holiday', 'days_until_next_holiday',
               'prev_holiday_name', 'next_holiday_name']] = (
    outflow_train['posted_date']
    .parallel_apply(lambda x: holiday_context(x, all_holidays))
)

# Optional: fill NaNs if outside holiday range
# outflow_train[['days_since_prev_holiday', 'days_until_next_holiday']] = outflow_train[
#     ['days_since_prev_holiday', 'days_until_next_holiday']
# ].fillna(999)


INFO: Pandarallel will run on 64 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:
df

NameError: name 'df' is not defined

In [ ]:
import matplotlib.pyplot as plt
plt.hist(outflow_train['amount'], bins = 1000)
plt.show()
plt.hist(outflow_train['log_amnt'], bins = 1000)
plt.show()